In [1]:
from datasets import load_dataset

TG_data = load_dataset("skg/toxigen-data", name="train", use_auth_token=True) # 250k training examples
TG_annotations = load_dataset("skg/toxigen-data", name="annotated", use_auth_token=True) # Human study

c:\Users\Alexandre\miniconda3\envs\huggingface\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Alexandre\miniconda3\envs\huggingface\lib\site-packages\datasets\load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [1]:
import pandas as pd
import csv

In [ ]:
dataset = pd.DataFrame(TG_data["train"])

In [3]:
dataset["masked"] = ""
dataset["nsubj"] = ""
dataset["preds"] = ""

In [2]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [3]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='roberta-large', tokenizer='roberta-large')

c:\Users\Alexandre\miniconda3\envs\huggingface\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
for group in ['middle_east']:

    dataset_temp = dataset.loc[dataset['group'] == group]

    for index, row in dataset_temp.iterrows():
        cnt = 0
        # Access row values using row['column_name']
        doc = nlp(row['generation'])
        for t in doc:
            if t.dep_ == "nsubj" and cnt == 0:
                dataset_temp.at[index, "nsubj"] = t.text
                masked_str = dataset_temp.at[index, "generation"].replace(t.text, "<mask>", 1)
                dataset_temp.at[index, "masked"] = masked_str

                preds = []
                unmasked = []
                if len(masked_str) > 512:
                    try:
                        unmasked = unmasker(masked_str[:512])
                    except Exception as e:
                        print(e)
                        print(index)
                else:
                    try:
                        unmasked = unmasker(masked_str)
                    except Exception as e:
                        print(e)
                        print(index)

                for i in unmasked:
                    preds.append(i["token_str"])

                dataset_temp.at[index, "preds"] = preds

                cnt = 1

    dataset_temp.to_csv("toxigen_masked_pred_" + group + ".csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

In [4]:
import os
import pandas as pd
from tqdm.auto import tqdm

directory = os.fsencode("masked_data")

for file in tqdm(os.listdir(directory)):

    filename = os.fsdecode(file)
    dataset_temp = pd.read_csv("masked_data/" + filename)

    dataset_temp["masked_dobj"] = ""
    dataset_temp["dobj"] = ""
    dataset_temp["preds_dobj"] = ""

    for index, row in dataset_temp.iterrows():
        cnt = 0
        # Access row values using row['column_name']
        doc = nlp(row['generation'])
        for t in doc:
            if t.dep_ == "dobj" and cnt == 0:
                dataset_temp.at[index, "dobj"] = t.text
                masked_str = dataset_temp.at[index, "generation"].replace(t.text, "<mask>", 1)
                dataset_temp.at[index, "masked_dobj"] = masked_str

                preds = []
                unmasked = []
                if len(masked_str) > 512:
                    try:
                        unmasked = unmasker(masked_str[:512])
                    except Exception as e:
                        print(e)
                        print(index)
                else:
                    try:
                        unmasked = unmasker(masked_str)
                    except Exception as e:
                        print(e)
                        print(index)

                for i in unmasked:
                    preds.append(i["token_str"])

                dataset_temp.at[index, "preds_dobj"] = preds

                cnt = 1

    dataset_temp.to_csv("masked_data/" + filename, index=False, quoting=csv.QUOTE_NONNUMERIC)

100%|██████████| 8/8 [2:21:19<00:00, 1059.90s/it]  
